# Library dependencies

`opu_analysis_lib` needs below library to funcion properly:

* numpy
* scipy
* sklearn
* mpllayout, this should be included with the original distribution but you can also get it from https://github.com/lguangyu/MatplotlibLayout.git
* skfeature, install with `pip install skfeature-chappers`, then fix a bug manually (https://github.com/charliec443/scikit-feature/issues/11). Or, just install from my patched version https://github.com/lguangyu/scikit-feature.git.

In [ ]:
%matplotlib inline

import opu_analysis_lib as oal
import os

# Dataset preparation

To begin OPU analysis, first we need to configure the dataset we need to use in the analysis. It's genuinly a list with each dataset's information stored in dicts. Using the mock datasets as an example:

In [ ]:
dataset_config = [
	{
		# name of the dataset (biosample) shown in outputs
		"name": "MOCK_0",
		# color of the dataset (biosample) shown in figure outputs
		"color": "#66a61e",
		# the spectral data, a table with:
		#	- the 1st row is wavenumbers
		#	- the rest rows are intensities
		#	- with/without first column as sample names (see below)
		"file": "example.assets/data/mock_00.tsv",
	},
	{
		"name": "MOCK_1",
		"color": "#e6ab02",
		"file": "example.assets/data/mock_01.tsv",
	},
	{
		"name": "MOCK_2",
		"color": "#e41a1c",
		# file can also be a list of files
		"file": [
			"example.assets/data/mock_02-1.tsv",
			"example.assets/data/mock_02-2.tsv",
		]
	},
]

# now we can create the OPU analyzer object
opu_analysis = oal.OPUAnalysis.from_config(dataset_config,
	# we also need extra reconcile parameters to tell the program how to
	# preprocess the data, and more importantly, if need binning to force align
	# the wavenumbers in each dataset (as they can be slightly different each
	# run)
	reconcile_param=dict(
		# set to True if the dataset files contain first column as sample sames
		with_spectra_names=False,
		# use binning to re-align wavenumbers, 5 is a typical window size
		bin_size=5.0,
		# the wavenumber range to extract here 400-1800
		wavenum_low=400,
		wavenum_high=1800,
		# normalization method, optional depending OPU clustering metric
		normalize="l2",
	)
)

# as an alternative, we can use `oal.OPUAnalysis.from_config.json()` to read
# above config settings if it's saved as a json.

# OPU identification and output

## Hierarchical clustering analysis

OPUs are recognized by running hierarchical clustering analysis (HCA) on the spectra, done by calling `run_hca()` method. Possible parameters are:

### `metric`

The metric used to calculate distances between spectra, can be:

* cosine: the cosine dissimilarity, this is the default
* sqrt_cosine: cosine dissimilarity taken square root; unlike cosine dissimilarity which does not sufficice triangular inequality, sqrt_cosine is a true metric
* euclidean: the Euclidean distance

### `cutoff`

The distance cutoff to split clusters, can be any real number, e.g. `cutoff=0.7`; in addition, this parameter can take two other str values to use built-in automated cutoff optimizer, to find a most balanced cutoff regarding to model complexity and number of clusters:

* `cutoff="aic"`: using Akaike information criterion
* `cutoff="bic"`: using Bayesian information criterion

The default is 0.7.

### `max_n_opus`

The maximum of top-sized OPUs reported; can be any non-negative integers; if set to 0, all OPUs will be reported without this limit.

### `opu_min_size`

The minimum number of spectra within a cluster to report this cluster as an OPU. This option can filter out small clusters (usually singletons). It accepts any non-negative integer or a fraction value between 0-1:

* integer: the minimum number of spectra
* fraction (0-1): the fraction of total number of spectra analyzed; e.g. if the total number of spectra is 103, and `opu_min_size=0.1`, then only clusters with 11 spectra or more will be reported OPUs.

In [ ]:
opu_analysis.run_hca(metric="cosine", cutoff=0.7, max_n_opus=0,
	opu_min_size=0.1)

## Save per spectra OPU labels

`save_opu_labels()` will save all OPU labels into a two-column text file, with the first column as the spectra name and the second column is OPU labels; Note that for those spectra belong to clusters failed to be reported as OPUs, their label will be shown as '-' instead of a numerical value. Parameters are:

### first argument

The name of output file.

### `delimiter`

The column delimiter in output file, default is `<tab>`.

In [ ]:
opu_analysis.save_opu_labels("example.assets/opu_labels.txt", delimiter="\t")

## Save spectra data in each reported OPU

`save_opu_collections()` will save the spectra in each OPUs; this function can generate multiple outputs, each output corresponds to an OPU with parameter:

### `prefix`

The prefix of generated files; for example, if `prefix="spectra_collection"`, the generated files will be `spectra_collection.OPU_00.txt`, `spectra_collection.OPU_01.txt`, etc. The `prefix` can contain folder(s) so that the output files will be organized into a folder.

In [ ]:
opu_analysis.save_opu_collections("example.assets/spectra_collection")

## HCA clustering heatmap and dendrogram plot

After calling `run_hca()`, a heatmap and dendrogram showing the details of the OPU clustering can be plotted by calling method `plot_opu_hca()`. Parameters are as follows:

### `plot_to`

How the plot will be shown, possible values are:

* `plot="show"`: show the plot in `matplotlib`'s interactive mode (only works in command-line)
* `plot="jupyter"`: embed the plot into Jupyter notebook (only works in jupyter notebook)
* `plot="plot.png"`: save as an image; `plot.png` can be changed to arbitrary file name of supported format, e.g. `my_image.tiff`;

### `dpi`

The resolution of output image, default is 300. Higher the dpi, larger the image file, and slower the rendering time. Typically 300 is good overall, <=150 has better portability and 600 is good for publication/print. Please note that some journal may require a minimal resolution in author's guide.


In [ ]:
opu_analysis.plot_opu_hca(plot_to="jupyter", dpi=300)

# OPU abundance analysis

Abundance analysis can be done in two ways:

* `plot_opu_abundance_stackbar()`: abundances in each biosample as stacked bar plot
* `plot_opu_abundance_biplot()`: PCA-like sample similarity analysis based on OPU abundance profiles

## Per biosample OPU abundance stackbar plot

`plot_opu_abundance_stackbar()` outputs a stacked barplot showing the OPU relative abundances across different bioasmple, accepts two parameters:

### `plot_to`

How the plot will be shown, possible values are:

* `plot="show"`: show the plot in `matplotlib`'s interactive mode (only works in command-line)
* `plot="jupyter"`: embed the plot into Jupyter notebook (only works in jupyter notebook)
* `plot="plot.png"`: save as an image; `plot.png` can be changed to arbitrary file name of supported format, e.g. `my_image.tiff`;

### `dpi`

The resolution of output image, default is 300. Higher the dpi, larger the image file, and slower the rendering time. Typically 300 is good overall, <=150 has better portability and 600 is good for publication/print. Please note that some journal may require a minimal resolution in author's guide.

In [ ]:
opu_analysis.plot_opu_abundance_stackbar(plot_to="jupyter", dpi=300)

## Decomposition biplot

PCA-like data decomposition and biosample similarity biplot, `plot_opu_abundance_plot()` accepts three parameters:

### `plot_to`

How the plot will be shown, possible values are:

* `plot="show"`: show the plot in `matplotlib`'s interactive mode (only works in command-line)
* `plot="jupyter"`: embed the plot into Jupyter notebook (only works in jupyter notebook)
* `plot="plot.png"`: save as an image; `plot.png` can be changed to arbitrary file name of supported format, e.g. `my_image.tiff`;

### `dpi`

The resolution of output image, default is 300. Higher the dpi, larger the image file, and slower the rendering time. Typically 300 is good overall, <=150 has better portability and 600 is good for publication/print. Please note that some journal may require a minimal resolution in author's guide.

### `method`

The method to decomposite data, default is `method="pca"`. Currently only supports PCA.

In [ ]:
opu_analysis.plot_opu_abundance_biplot(plot_to="jupyter", dpi=300, method="pca")

# OPU-based feature ranking

Features that provide efficient information distinguishing recognized OPUs can be ranked. One-versus-the-rest (OVR) analysis is applied to assess the individual feature ranking at per OPU basis, therefore identifying the most important features in distinguishing each recognized OPU from other OPUs.

## Feature ranking analysis

`rank_features()` runs the OVR feature ranking, accepts one parameter:

### `method`

The ranking score used in the analysis, value can be:

* `method="fisher_score"`: Fisher score
* `method="lap_score"`: Laplacian score
* `method="trace_ratio"`: trace-ratio

In [ ]:
opu_analysis.rank_features("fisher_score")  # 'method=' can be omitted

## Save ranked feature table

`save_opu_feature_rank_table()` saves the sorted features (wavenumbers) in each OPU in descending order, i.e. the first feature on the list is the most imporant feature to distinguish the corresponding OPU from others, and the last feature is the least important. Possible parameters are:

### first argument

Path to the output file

### `delimiter`

The delimiter used in tabular output, default is `<tab>`.

In [ ]:
opu_analysis.save_opu_feature_rank_table("example.assets/opu.feature_rank.txt", delimiter="\t")

## Make OPU feature rank plot

`plot_opu_feature_rank()` will visualize the feature rank and output as a figure. Similar to other plot functions, it can accept two common arguments:

### `plot_to`

How the plot will be shown, possible values are:

* `plot="show"`: show the plot in `matplotlib`'s interactive mode (only works in command-line)
* `plot="jupyter"`: embed the plot into Jupyter notebook (only works in jupyter notebook)
* `plot="plot.png"`: save as an image; `plot.png` can be changed to arbitrary file name of supported format, e.g. `my_image.tiff`;

### `dpi`

The resolution of output image, default is 300. Higher the dpi, larger the image file, and slower the rendering time. Typically 300 is good overall, <=150 has better portability and 600 is good for publication/print. Please note that some journal may require a minimal resolution in author's guide.

In [ ]:
opu_analysis.plot_opu_feature_rank(plot_to="jupyter", dpi=300)